# 양정고등학교 HRD 메이커 퀀트 실습 4
## 주요 재무지표 수집 및 가공하기
[인증키 신청](https://opendart.fss.or.kr/uss/umt/EgovMberInsertView.do)


### 1. 초기 세팅

In [1]:
! mkdir data

In [ ]:
! wget https://raw.githubusercontent.com/stockjeong/hrdquant/refs/heads/main/%EC%A2%85%EB%AA%A9%EC%A0%95%EB%B3%B42025.csv # 샘플 데이터 받아오기

--2025-04-25 06:49:10--  https://github.com/stockjeong/hrdquant/2025stock.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-04-25 06:49:10 ERROR 404: Not Found.



In [ ]:
! pip install OpenDartReader -q

### 2. 데이터 파싱 전략 수립

In [ ]:
import OpenDartReader
my_api = "49526a379265e67332f8abb3ddc10ab9f6af07db"
dart = OpenDartReader(my_api)

In [ ]:
import pandas as pd

stock_list = pd.read_csv(
    "./2024stock.csv",
    encoding="utf-8",
    sep="\t",
    usecols=["Name", "Code"],
    dtype=str)

In [ ]:
report = dart.finstate("005380", 2023)
display(report[["fs_nm", "account_nm", "thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]])

,fs_nm,account_nm,thstrm_amount,frmtrm_amount,bfefrmtrm_amount
0,연결재무제표,유동자산,"101,724,717,000,000","96,389,273,000,000","88,565,366,000,000"
1,연결재무제표,비유동자산,"180,738,638,000,000","159,353,189,000,000","145,381,049,000,000"
2,연결재무제표,자산총계,"282,463,355,000,000","255,742,462,000,000","233,946,415,000,000"
3,연결재무제표,유동부채,"73,362,103,000,000","74,236,472,000,000","64,236,787,000,000"
4,연결재무제표,비유동부채,"107,291,812,000,000","90,609,445,000,000","87,093,839,000,000"
5,연결재무제표,부채총계,"180,653,915,000,000","164,845,917,000,000","151,330,626,000,000"
6,연결재무제표,자본금,"1,488,993,000,000","1,488,993,000,000","1,488,993,000,000"
7,연결재무제표,이익잉여금,"88,665,805,000,000","79,953,601,000,000","73,167,855,000,000"
8,연결재무제표,자본총계,"101,809,440,000,000","90,896,545,000,000","82,615,789,000,000"
9,연결재무제표,매출액,"162,663,579,000,000","142,151,469,000,000","116,448,159,000,000"


### 3. 데이터 파싱

#### 데이터 파싱 함수 작성

In [ ]:
import numpy as np
import pandas as pd

def find_fins_ind_list(stock_code, stock_name, year, ind_list):
    try: # 데이터 가져오기
        report = None
        report = dart.finstate(stock_code, year)
    except:
        pass

    if report is None:  # 리포트가 없다면 (참고: 리포트가 없으면 None을 반환함)
        # 리포트가 없으면 당기, 전기, 전전기 값 모두 제거
        data = [[stock_name, year] + [np.nan] * len(ind_list)]
        data.append([stock_name, year - 1] + [np.nan] * len(ind_list))
        data.append([stock_name, year - 2] + [np.nan] * len(ind_list))

    else:
        report = report[report["account_nm"].isin(ind_list)]  # 관련 지표로 필터링
        if sum(report["fs_nm"] == "연결재무제표") > 0:
            # 연결재무제표 데이터가 있으면 연결재무제표를 사용
            report = report.loc[report["fs_nm"] == "연결재무제표"]
        else:
            # 연결재무제표 데이터가 없으면 일반재무제표를 사용
            report = report.loc[report["fs_nm"] == "재무제표"]
        data = []
        for y, c in zip([year, year - 1, year - 2],
                        ["thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]):
            record = [stock_name, y]
            for ind in ind_list:
                # account_nm이 indic인 행의 c 컬럼 값을 가져옴
                if sum(report["account_nm"] == ind) > 0:
                    value = report.loc[report["account_nm"] == ind, c].iloc[0]
                else:
                    value = np.nan
                record.append(value)
            data.append(record)

    return pd.DataFrame(data, columns=["기업", "연도"] + ind_list)

In [ ]:
ind_list = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익']
display(find_fins_ind_list("005930", "삼성전자", 2020, ind_list))

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
0,삼성전자,2020,"378,235,718,000,000","102,287,702,000,000","275,948,016,000,000","236,806,988,000,000","35,993,876,000,000","26,407,832,000,000"
1,삼성전자,2019,"352,564,497,000,000","89,684,076,000,000","262,880,421,000,000","230,400,881,000,000","27,768,509,000,000","21,738,865,000,000"
2,삼성전자,2018,"339,357,244,000,000","91,604,067,000,000","247,753,177,000,000","243,771,415,000,000","58,886,669,000,000","44,344,857,000,000"


#### 데이터 수집

In [ ]:
import time
data = pd.DataFrame() # 이 데이터프레임에 각각의 데이터를 추가할 예정
for code, name in stock_list[['Code', 'Name']].values:
    print(name)
    for year in [2023, 2024]:
        try:
            result = find_fins_ind_list(code, name, year, ind_list) # 재무지표 데이터
        except:
            pass

        data = pd.concat([data, result], axis = 0, ignore_index = True) # data에 부착
        time.sleep(0.5)

삼성전자
SK하이닉스
LG에너지솔루션
삼성바이오로직스
삼성전자우
현대차
기아
셀트리온
POSCO홀딩스
NAVER
삼성SDI
LG화학
삼성물산
KB금융
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

현대모비스
신한지주
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

카카오
포스코퓨처엠
삼성생명
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

하나금융지주
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

메리츠금융지주
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

LG전자
에코프로
삼성화재
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

한국전력
한미반도체
SK
LG
카카오뱅크
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

크래프톤
삼성에스디에스
삼성전기
KT&G
SK이노베이션
HLB
SK텔레콤
두산에너빌리티
HD현대중공업
한화에어로스페이스
기업은행
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

HMM
SK스퀘어
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

우리금융지주
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

고려아연
S-Oil
하이브
KT
포스코인터내셔널
HD한국조선해양
한화오션
에코프로머티
아모레퍼시픽
대한항공
삼성중공업
HD현대일렉트릭
SK바이오팜
DB손해보험
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

한국타이어앤테크놀로지
포스코DX
금양
현대글로비스
엔켐
엘앤에프
LG생활건강
유한양행
HD현대
맥쿼리인프라
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'mes

### 4. 데이터 정제

In [ ]:
data.drop_duplicates(inplace = True)
data.sort_values(by = ['기업', '연도'], inplace = True)

In [ ]:
display(data.head())

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
6788,3S,2020,"49,658,374,621","16,293,352,322","33,365,022,299","22,592,015,218","467,181,073","-1,080,358,319"
6787,3S,2021,"54,614,752,349","18,068,427,911","36,546,324,438","23,477,908,383","-1,408,265,950","695,334,883"
6786,3S,2022,"65,127,864,527","26,788,875,680","38,338,988,847","27,110,628,415","1,452,134,065","1,171,369,995"
6789,3S,2023,"67,635,499,845","21,680,745,544","45,954,754,301","41,857,755,025","2,180,508,808","1,431,602,721"
4928,AJ네트웍스,2020,"1,588,170,350,000","1,300,162,539,894","288,007,810,106","871,956,093,147","21,089,140,851","-3,316,812,122"


In [ ]:
# 숫자로 모두 변환
def convert_str_to_float(value):
    if type(value) == float: # nan의 자료형은 float임
        return value
    elif value == '-': # -로 되어 있으면 0으로 변환
        return 0
    else:
        return float(value.replace(',', ''))

for ind in ind_list:
    data[ind] = data[ind].apply(convert_str_to_float)

In [ ]:
display(data.head())

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
6788,3S,2020,4.965837e+10,1.629335e+10,3.336502e+10,2.259202e+10,4.671811e+08,-1.080358e+09
6787,3S,2021,5.461475e+10,1.806843e+10,3.654632e+10,2.347791e+10,-1.408266e+09,6.953349e+08
6786,3S,2022,6.512786e+10,2.678888e+10,3.833899e+10,2.711063e+10,1.452134e+09,1.171370e+09
6789,3S,2023,6.763550e+10,2.168075e+10,4.595475e+10,4.185776e+10,2.180509e+09,1.431603e+09
4928,AJ네트웍스,2020,1.588170e+12,1.300163e+12,2.880078e+11,8.719561e+11,2.108914e+10,-3.316812e+09


### 5. 주요 지표 계산

#### 부채비율

In [ ]:
data['부채비율'] = data['부채총계'] / data['자본총계'] * 100
display(data['부채비율'].head())

6788     48.833632
6787     49.439795
6786     69.873715
6789     47.178460
4928    451.433084
Name: 부채비율, dtype: float64

#### 매출액 증가율, 영업이익 증가율, 당기순이익 증가율

In [ ]:
data['매출액증가율'] = (data['매출액'].diff() / data['매출액'].shift(1)) * 100
data.loc[data['연도'] == 2013, '매출액증가율'] = np.nan

In [ ]:
data['영업이익증가율'] = (data['영업이익'].diff() / data['영업이익'].shift(1)) * 100
data.loc[data['연도'] == 2013, '영업이익증가율'] = np.nan

data['당기순이익증가율'] = (data['당기순이익'].diff() / data['당기순이익'].shift(1)) * 100
data.loc[data['연도'] == 2013, '당기순이익증가율'] = np.nan

In [ ]:
data = data.replace({np.inf:np.nan, -np.inf: np.nan})

#### 매출액 상태, 영업이익 상태, 당기순이익 상태

In [ ]:
# 상태를 나타내는 함수 정의
def add_state(data, col):
    data[col + "_상태"] = np.nan # 상태를 결측으로 초기화
    value = data[col].values
    cur_value = value[1:]
    pre_value = value[:-1]
    # 흑자지속
    cond1 = (cur_value > 0) & (pre_value > 0)
    cond1 = np.insert(cond1, 0, np.nan)
    # 적자지속
    cond2 = (cur_value <= 0) & (pre_value <= 0)
    cond2 = np.insert(cond2, 0, np.nan)
    # 흑자전환
    cond3 = (cur_value > 0) & (pre_value <= 0)
    cond3 = np.insert(cond3, 0, np.nan)
    # 적자전환
    cond4 = (cur_value <= 0) & (pre_value > 0)
    cond4 = np.insert(cond4, 0, np.nan)

    # 조건에 따른 변환
    data.loc[cond1, col + "_상태"] = "흑자지속"
    data.loc[cond2, col + "_상태"] = "적자지속"
    data.loc[cond3, col + "_상태"] = "흑자전환"
    data.loc[cond4, col + "_상태"] = "적자전환"

In [ ]:
add_state(data, "매출액")
add_state(data, "영업이익")
add_state(data, "당기순이익")

#### ROA

In [ ]:
data['ROA'] = (data['당기순이익'] / data['자산총계']) * 100

#### ROE

In [ ]:
average_equity = data['자본총계'].rolling(2).mean() # 평균 자기 자본
data['ROE'] = (data['당기순이익'] / average_equity) * 100
data.loc[data['연도'] == 2013, 'ROE'] = np.nan

#### 데이터 저장

In [ ]:
data.to_csv("./data/주요재무지표_수정.csv", index = False, encoding = "utf-8")
display(data)

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,부채비율,매출액증가율,영업이익증가율,당기순이익증가율,매출액_상태,영업이익_상태,당기순이익_상태,ROA,ROE
6788,3S,2020,4.965837e+10,1.629335e+10,3.336502e+10,2.259202e+10,4.671811e+08,-1.080358e+09,48.833632,NaN,NaN,NaN,적자전환,적자전환,적자전환,-2.175581,NaN
6787,3S,2021,5.461475e+10,1.806843e+10,3.654632e+10,2.347791e+10,-1.408266e+09,6.953349e+08,49.439795,3.921267,-401.438999,-164.361506,흑자지속,적자전환,흑자전환,1.273163,1.989190
6786,3S,2022,6.512786e+10,2.678888e+10,3.833899e+10,2.711063e+10,1.452134e+09,1.171370e+09,69.873715,15.472929,-203.115045,68.461273,흑자지속,흑자전환,흑자지속,1.798570,3.128437
6789,3S,2023,6.763550e+10,2.168075e+10,4.595475e+10,4.185776e+10,2.180509e+09,1.431603e+09,47.178460,54.396108,50.158919,22.216100,흑자지속,흑자지속,흑자지속,2.116644,3.396700
4928,AJ네트웍스,2020,1.588170e+12,1.300163e+12,2.880078e+11,8.719561e+11,2.108914e+10,-3.316812e+09,451.433084,1983.141087,867.166047,-331.685235,흑자지속,흑자지속,적자전환,-0.208845,-1.986338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9117,희림,2023,1.935073e+11,1.202178e+11,7.328949e+10,2.286523e+11,7.376355e+09,6.377486e+09,164.031484,3.431106,-27.162720,-10.696738,흑자지속,흑자지속,흑자지속,3.295734,9.023100
10964,힘스,2020,9.086640e+10,1.312457e+10,7.774182e+10,NaN,2.496817e+10,1.720465e+10,16.882255,NaN,238.489349,169.771695,NaN,흑자지속,흑자지속,18.934010,22.782895
10963,힘스,2021,8.430302e+10,1.038679e+10,7.391623e+10,NaN,-4.371187e+09,-4.413924e+08,14.052104,NaN,-117.507035,-102.565541,NaN,적자전환,적자전환,-0.523578,-0.582089
10962,힘스,2022,7.952340e+10,1.758886e+10,6.193453e+10,NaN,-1.327548e+10,-8.645207e+09,28.399120,NaN,203.704340,1858.621469,NaN,적자지속,적자지속,-10.871275,-12.727505
